In [1]:
%%time
import pandas as pd
df = pd.read_csv('HMAHCC_COMP.csv')

<string>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 34.1 s


##### Summary
1. Opioid drugs have 77 brand names, and 213 generic names
2. People who use stronger drugs on day 0 are more likely to get LTOT

##### Next Step
1. Build features based on usage history of 213 generic names. 
2. The entries of these features can be binary numbers, total dosage or usage frequency.

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

In [3]:
rx_paid = df[df['event_descr']=='RX Claim - Paid']

rx_paid = rx_paid.drop(['event_attr2',
                        'event_attr7'],
                         axis = 1)

rx_paid.columns = ['id',
                   'event_descr',
                   'gpi_drug_class_description',
                   'rx_cost',
                   'net_paid_amount',
                   'brand_name',
                   'drug_group_description',
                   'generic_name',
                   'member_responsible_amount',
                   'gpi_drug_group8_id',
                   'Days',
                   'PAY_DAY_SUPPLY_CNT',
                   'PAYABLE_QTY',
                   'MME',
                   'DRUG_TYPE',
                   'Specialty',
                   'Specialty2',
                   'Specialty3']

In [4]:
# read in response_variable
response_variable = pd.read_csv('response_variable.csv')
response_variable = response_variable.iloc[:,:2]
response_variable.head()

,id,LTOT
0,ID10010854159,False
1,ID10013863216,True
2,ID10024447278,True
3,ID1002482139,True
4,ID1003386406,False


In [5]:
opioid_col = ['id',
              'gpi_drug_class_description',
              'brand_name',
              'drug_group_description',
              'generic_name',
              'gpi_drug_group8_id',
              'PAY_DAY_SUPPLY_CNT',
              'Specialty',
              'Specialty2',
              'Specialty3',
              'Days']

opioid = rx_paid[ (rx_paid['PAY_DAY_SUPPLY_CNT'].notnull())][opioid_col]

In [6]:
len(opioid['brand_name'].value_counts())

77

In [7]:
len(opioid['generic_name'].value_counts())

213

In [8]:
# top 5 most used brands
opioid['brand_name'].value_counts().head()

HYDROCODONE/ACETAMINOPHEN    86388
TRAMADOL HCL                 62296
OXYCODONE/ACETAMINOPHEN      33180
OXYCODONE HCL                19977
MORPHINE SULFATE ER           9850
Name: brand_name, dtype: int64

#### My hypothesis is that ppl who take stronger drug on day 0 are more likely to get LTOT

I look at HYDROCODONE/ACETAMINOPHEN first, because it's the most used opioid drug in this dataset

In [9]:
HYD = opioid[ (opioid['brand_name'] == 'HYDROCODONE/ACETAMINOPHEN') & (opioid['Days'] == 0)]
HYD['generic_name'].value_counts()
# there are 3 different dosages for HYD

HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG      3107
HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG    1204
HYDROCODONE-ACETAMINOPHEN TAB 10-325 MG     1140
Name: generic_name, dtype: int64

In [10]:
HYD2 = pd.merge(HYD, response_variable, on = 'id')
HYD2.head()

,id,gpi_drug_class_description,brand_name,drug_group_description,generic_name,gpi_drug_group8_id,PAY_DAY_SUPPLY_CNT,Specialty,Specialty2,Specialty3,Days,LTOT
0,ID10036289882,OPIOID COMBINATIONS,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG,65991702.0,2.0,Emergency,Emergency Medicine Emergency Med Svcs,HOSPICE & PALLIATIVE MEDICINE,0,False
1,ID10036954381,OPIOID COMBINATIONS,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG,65991702.0,2.0,Orthopaedic Surgery,Orthopaedic Surgery Hand Surgery,NaN,0,False
2,ID10055739763,OPIOID COMBINATIONS,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG,65991702.0,5.0,Emergency,Family Practice,Emergency Medicine Emergency Med Svcs,0,True
3,ID10074598346,OPIOID COMBINATIONS,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG,65991702.0,5.0,STUDENT IN ORGANIZED HEALTHCARE EDU/TRAI,NaN,NaN,0,False
4,ID10081072715,OPIOID COMBINATIONS,HYDROCODONE/ACETAMINOPHEN,PAIN,HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG,65991702.0,5.0,Emergency Medicine Undersea/Hyperbaric,Emergency,Emergency Medicine Emergency Med Svcs,0,False


In [11]:
(HYD2.groupby(by=['generic_name', 'LTOT'])['LTOT'].count().unstack().T/HYD['generic_name'].value_counts()).T

LTOT,False,True
HYDROCODONE-ACETAMINOPHEN TAB 10-325 MG,0.297368,0.702632
HYDROCODONE-ACETAMINOPHEN TAB 5-325 MG,0.670743,0.329257
HYDROCODONE-ACETAMINOPHEN TAB 7.5-325 MG,0.537375,0.462625


Next I looked at other drug to further support my view

In [12]:
OXY = opioid[ (opioid['brand_name'] == 'OXYCODONE/ACETAMINOPHEN') & (opioid['Days'] == 0)]
OXY['generic_name'].value_counts()

OXYCODONE W/ ACETAMINOPHEN TAB 5-325 MG      1049
OXYCODONE W/ ACETAMINOPHEN TAB 10-325 MG      388
OXYCODONE W/ ACETAMINOPHEN TAB 7.5-325 MG     184
OXYCODONE W/ ACETAMINOPHEN TAB 2.5-325 MG       1
Name: generic_name, dtype: int64

In [13]:
OXY = pd.merge(OXY, response_variable, on = 'id')
OXY.head()

,id,gpi_drug_class_description,brand_name,drug_group_description,generic_name,gpi_drug_group8_id,PAY_DAY_SUPPLY_CNT,Specialty,Specialty2,Specialty3,Days,LTOT
0,ID10024447278,OPIOID COMBINATIONS,OXYCODONE/ACETAMINOPHEN,PAIN,OXYCODONE W/ ACETAMINOPHEN TAB 5-325 MG,65990002.0,3.0,Emergency,Emergency Medicine Emergency Med Svcs,Family Practice,0,True
1,ID1013630023,OPIOID COMBINATIONS,OXYCODONE/ACETAMINOPHEN,PAIN,OXYCODONE W/ ACETAMINOPHEN TAB 10-325 MG,65990002.0,15.0,Internal Medicine,Hospitalist,Family Practice,0,True
2,ID10166893764,OPIOID COMBINATIONS,OXYCODONE/ACETAMINOPHEN,PAIN,OXYCODONE W/ ACETAMINOPHEN TAB 10-325 MG,65990002.0,30.0,Family Practice,General Practice,NaN,0,True
3,ID1020688098,OPIOID COMBINATIONS,OXYCODONE/ACETAMINOPHEN,PAIN,OXYCODONE W/ ACETAMINOPHEN TAB 5-325 MG,65990002.0,4.0,Obstetrics & Gynecology,Obstetrics & Gynecology Gynecology,Obstetrics & Gynecology Obstetrics,0,False
4,ID10230809175,OPIOID COMBINATIONS,OXYCODONE/ACETAMINOPHEN,PAIN,OXYCODONE W/ ACETAMINOPHEN TAB 10-325 MG,65990002.0,30.0,Anesthesiology,NaN,NaN,0,True


In [14]:
(OXY.groupby(by=['generic_name', 'LTOT'])['LTOT'].count().unstack().T/OXY['generic_name'].value_counts()).T

LTOT,False,True
OXYCODONE W/ ACETAMINOPHEN TAB 10-325 MG,0.195876,0.804124
OXYCODONE W/ ACETAMINOPHEN TAB 2.5-325 MG,1.000000,NaN
OXYCODONE W/ ACETAMINOPHEN TAB 5-325 MG,0.668255,0.331745
OXYCODONE W/ ACETAMINOPHEN TAB 7.5-325 MG,0.369565,0.630435
